In [1]:
import numpy as np
import pandas as pd

### Create student-to-lottery number and lottery number-to-student indices

In [2]:
student_rankings = np.load('Data/Generated/student_rankings.npy', allow_pickle=True).item()
school_rankings = np.load('Data/Generated/school_rankings.npy', allow_pickle=True).item()
student_demographics = np.load('Data/Generated/student_demographics.npy', allow_pickle=True)

In [3]:
id_index = 0
lottery_index = 17
to_lottery_numbers = {}
from_lottery_numbers = {}

for student in student_demographics:
	to_lottery_numbers[student[id_index]] = student[lottery_index]
	from_lottery_numbers[student[lottery_index]] = student[id_index]

np.save('Data/Generated/student_to_lottery_nums.npy', to_lottery_numbers)
np.save('Data/Generated/lottery_nums_to_student.npy', from_lottery_numbers)

### Create student id-based rankings

In [4]:
school_rankings_id = {
	school: [from_lottery_numbers[student] for student in ranking] for school, ranking in school_rankings.items()
}

In [5]:
np.save('Data/Generated/school_rankings_id.npy', school_rankings_id)

### Get school capacity

In [6]:
df = pd.read_excel('Data/school_directory.xlsx', sheet_name='Data')

cols = ['dbn', 'school_name'] + ['seats9' + student_type + str(i) for student_type in ['ge', 'swd'] for i in range(1, 13)]
df_selected = df[cols]

In [7]:
df_selected['overall_seats'] = df_selected.iloc[:, -25:-1].sum(axis=1)
df_selected = df_selected[df_selected['overall_seats'] != 0]
df_selected = df_selected[['dbn', 'overall_seats']]
df_selected

C:\Users\Tommaso\AppData\Local\Temp\ipykernel_21164\875658289.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_selected['overall_seats'] = df_selected.iloc[:, -25:-1].sum(axis=1)
C:\Users\Tommaso\AppData\Local\Temp\ipykernel_21164\875658289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['overall_seats'] = df_selected.iloc[:, -25:-1].sum(axis=1)


,dbn,overall_seats
0,01M292,90.0
1,01M448,105.0
2,01M450,95.0
3,01M458,28.0
4,01M515,60.0
...,...,...
441,32K545,126.0
442,32K549,115.0
443,32K552,113.0
444,32K554,62.0


In [8]:
school_codes = df_selected['dbn'].values
schools = list(school_rankings.keys())
not_in_df = [school for school in schools if school not in school_codes]
new_rows = pd.DataFrame({'dbn': not_in_df})
new_rows

,dbn
0,13K963
1,15K960
2,08X636
3,28Q686
4,10X228


In [9]:
df_selected = pd.concat((df_selected, new_rows))
df_selected

,dbn,overall_seats
0,01M292,90.0
1,01M448,105.0
2,01M450,95.0
3,01M458,28.0
4,01M515,60.0
...,...,...
0,13K963,NaN
1,15K960,NaN
2,08X636,NaN
3,28Q686,NaN


In [10]:
seat_distribution = df_selected['overall_seats'].replace(0, np.nan, inplace=False)
low, hi = seat_distribution.min(), seat_distribution.max()
mean = seat_distribution.mean()
stdev = seat_distribution.std()
missing = seat_distribution.value_counts(dropna=False)
print(f'Range: [{int(low)}, {int(hi)}]')
print(f'Distribution: {mean:.2f} +/- {stdev:.2f}')
print(f'Missing: {missing[np.nan]}')

Range: [10, 1060]
Distribution: 149.76 +/- 127.85
Missing: 5


In [11]:
df_selected['overall_seats'].replace(np.nan, int(mean), inplace=True)
df_selected

,dbn,overall_seats
0,01M292,90.0
1,01M448,105.0
2,01M450,95.0
3,01M458,28.0
4,01M515,60.0
...,...,...
0,13K963,149.0
1,15K960,149.0
2,08X636,149.0
3,28Q686,149.0


In [12]:
to_export = {}

for i in range(0, df_selected.size // 2):
	item = df_selected.iloc[i]
	to_export[item['dbn']] = int(item['overall_seats'])

np.save('Data/Generated/school_capacities.npy', to_export)

In [13]:
np.load('Data/Generated/school_capacities.npy', allow_pickle=True).item()

{'01M292': 90,
 '01M448': 105,
 '01M450': 95,
 '01M458': 28,
 '01M515': 60,
 '01M539': 150,
 '01M696': 125,
 '02M047': 69,
 '02M135': 115,
 '02M139': 108,
 '02M260': 114,
 '02M280': 112,
 '02M282': 125,
 '02M288': 108,
 '02M294': 81,
 '02M296': 129,
 '02M298': 160,
 '02M300': 108,
 '02M303': 107,
 '02M305': 100,
 '02M308': 95,
 '02M313': 25,
 '02M316': 120,
 '02M374': 204,
 '02M376': 118,
 '02M392': 108,
 '02M393': 129,
 '02M394': 60,
 '02M399': 134,
 '02M400': 300,
 '02M407': 61,
 '02M408': 124,
 '02M411': 120,
 '02M412': 91,
 '02M413': 108,
 '02M414': 124,
 '02M416': 125,
 '02M418': 160,
 '02M419': 100,
 '02M420': 442,
 '02M422': 108,
 '02M425': 276,
 '02M427': 108,
 '02M432': 90,
 '02M437': 118,
 '02M438': 68,
 '02M439': 100,
 '02M449': 110,
 '02M459': 85,
 '02M489': 200,
 '02M500': 95,
 '02M507': 108,
 '02M519': 176,
 '02M520': 240,
 '02M529': 200,
 '02M531': 60,
 '02M533': 109,
 '02M534': 113,
 '02M542': 135,
 '02M543': 125,
 '02M544': 30,
 '02M545': 108,
 '02M546': 120,
 '02M550'